### File Explanation

**loadData.ipynb:**
<br> This notebook is to load raw AddNeuroMed data from 'mendelianGenesX2_v3.Rdata' and turn into NumPy arrays and lists for computation advantage 

**Processes are as follows:**
<br> 1) Load all variables from 'mendelianX2_v3.Rdata'
<br> 2) Store most of them as NumPy arrays by using "rpy2" which converts R data into either NumPy or Pandas 
<br> 3) Store other variables (nnNs_lRlN, nPairs_lPlC) as lists
<br> 4) Load ".bed, .bim, .fam" files of AddNeuroMed
<br> 5) Retrieve SNP locations from .bim
<br> 6) Get gene start and end points by using **Biopython package's Entrez API**
<br> 7) Store necessary files in .csv format for MatrixEQTL and LD analysis
<br> 8) Save them in "loadData.pickle" format to be called by "preprocessData.ipynb"

**Some abbreviations used:**
<br> **S:** number of subjects 
<br> **D:** number of demographic category
<br> **G:** number of genes
<br> **R:** number of transcripts
<br> **N:** number of SNPs
<br> **P:** number of pairs
<br> **C:** number of categories 

**Variables created:**
<br> 1) **sDem_nSD:** Demographic information of subjects, a NumPy array, shape of (S x D) 
<br> 2) **sGenesAD_nG:** Genes, a NumPy array, shape of (G x 1)
<br> 3) **rRna_nSR:** RNA (gene) expression levels, a NumPy array, shape of (S x R)
<br> 4) **rSnp_nSN:** SNP encodings (1, 2, 3), a NumPy array, shape of (S x N)
<br> 5) **nNs_lRlN:** SNP names for corresponding genes on which SNPs are, a list, length of (R : [N]) 
<br> 6) **nPairs_lPlC:** Known gene pairs, a list, length of (2204) with columns for "TF ID, TF Entrez ID, TF Gene Symbol, Target Entrez ID, Target Gene Symbol"
<br> 7) **nSs:** Subject names
<br> 8) **nRs:** Transcript (gene) names
<br> 9) **nDs:** Demographic category names
<br> 10) **nNs:** SNPs names 

**Files created for MatrixEQTL + LD analysis:**
<br> 1) **eqtl_expression.csv**: Gene expressions of all transcripts, csv format (rows: R, columns: S)
<br> 2) **eqtl_genotype.csv**: SNP encodings (0, 1, 2) of SNPs, csv format (rows: N, columns: S)
<br> 3) **eqtl_probe_loc.csv**: Gene locations, csv format (rows: R, columns: gene ID, chromosome number, start location, end location)
<br> 4) **eqtl_snp_loc.csv**: SNP locations, csv format (rows: N, columns: SNP ID, chromosome number, position)
<br> 5) **eqtl_subjects.csv**: Subject names, csv format (rows: S, columns: 1)
<br> 6) **eqtl_snps.csv**: SNP names, csv format (rows: N, columns: 1)
<br> 7) **eqtl_genes.csv**: Transcript names, csv format (rows: R, columns: 1)
<br> 8) **eqtl_snp_gene.csv**: SNP to gene mapping, csv format (rows: N, columns: SNP ID, gene ID)
<br> 9) **eqtl_snp_alleles.csv**: SNP allele pairs, csv format (rows: N, columns: a0, a1 for minor and major allele)

In [1]:
from __future__ import print_function
import numpy as np
import tensorflow as tf
from tensorflow.contrib import rnn
import os

# Set up environment variables for rpy2 to work
os.environ['R_HOME'] = '/usr/local/Cellar/r/3.6.0_2/lib/R'
os.environ['R_USER'] = '/usr/local/lib/python3.7/site-packages/rpy2'

In [2]:
# Load Rdata with rpy2
import rpy2
from rpy2.robjects import r, pandas2ri
pandas2ri.activate()
# Clear the workspace
r( 'rm( list = ls( ) )' )
# 16GB memory limit
r( 'memory.limit( 16000 )' )
r['load']('mendelianGenesX2_v3.Rdata')
print( 'Data loaded from Rdata file.' )

# Hale the user
print( 'Variables loaded:' )
[ print( a ) for a in r['ls']() ];

Data loaded from Rdata file.
Variables loaded:
dem_iSiD
demRnaSnp_rc_iSiG
genesAD_iG
nnInput_jPiNiF
nNs_c5_jG
pairs_c5_PiC
rna_iSiR
rna_iSiR_p
snp_iSiN


In [3]:
## Dimensions
assert( all( r( 'dimnames( snp_iSiN )[[1]] == dimnames( rna_iSiR )[[1]]' ) ) )
assert( all( r( 'names( nNs_c5_jG ) == gsub( "rna", "", dimnames( rna_iSiR )[[2]] )' )  ) )
nSs = r( 'dimnames( rna_iSiR )[[1]]' ) 
nRs = r( 'dimnames( rna_iSiR )[[2]]' ) 
nDs = r( 'dimnames( dem_iSiD )[[2]]' ) 
nNs = r( 'dimnames( snp_iSiN )[[2]]' ) 

In [4]:
# TRANSFORM DATA INTO NUMPYS
# Transforms all the data from rpy2 objects into NumPys and lists

# Transform R objects into numpys
import numpy as np
## dem_iSiD
sDem_nSD = np.array( rpy2.robjects.r( 't( dem_iSiD )' ),
                    dtype = np.str )
sDem_nSD.shape = r( 'dim( dem_iSiD )' )
assert( all( sDem_nSD[0:10,10] == ( r( 'dem_iSiD[1:10,11]' ) ) ) )

## genesAD_iG
sGenesAD_nG = np.array( rpy2.robjects.r( 'genesAD_iG' ),
                    dtype = np.str )
sGenesAD_nG = np.transpose( sGenesAD_nG )
assert( all( sGenesAD_nG[0:10] == ( r( 'genesAD_iG[1:10]' ) ) )  ) 

## rna_iSiR
rRna_nSR = np.array( rpy2.robjects.r( 'rna_iSiR' ),
                     dtype = np.float )
assert( all( rRna_nSR[0:10,10] == ( r( 'rna_iSiR[1:10,11]' ) ) ) )

## snp_iSiN
rSnp_nSN = np.array( rpy2.robjects.r( 'snp_iSiN' ),
                     dtype = np.float )
rSnp_nSN.shape = r( 'dim( snp_iSiN )' )
assert( all( rSnp_nSN[0:100,10] == [ nSRN for nSRN in map(float, (r( 'snp_iSiN[1:100,11]' ) ) ) ] ) )

## nNs_c5_jG 
nNs_lRlN = [  nNs  for nNs in r( 'nNs_c5_jG') ]
assert(all( (nNs_lRlN[0][0:10]) == ( r( 'nNs_c5_jG[[1]][1:10]' ) ) ) )
assert(all( (nNs_lRlN[10][0:10]) == ( r( 'nNs_c5_jG[[11]][1:10]' ) ) ) )

## pairs_c5_PiC
nPairs_lPlC = [  nPairs  for nPairs in r( 'pairs_c5_PiC') ]
assert(all( (nPairs_lPlC[0][0:10]) == ( r( 'pairs_c5_PiC[[1]][1:10]') ) ) )
assert(all( (nPairs_lPlC[4][0:10]) == ( r( 'pairs_c5_PiC[[5]][1:10]' ) ) ) )

In [6]:
# Check data
assert( len( nNs_lRlN ) == rRna_nSR.shape[1] )
assert( sDem_nSD.shape[0] == rRna_nSR.shape[0] )
assert( sDem_nSD.shape[0] == rSnp_nSN.shape[0] )
print( 'Data transformed into numpys.' )

Data transformed into numpys.


In [8]:
# LOAD HBTRC STUDY DATA
from pandas_plink import read_plink1_bin, read_plink
import pandas as pd
# read_plink1_bin returns (dataArray in shape = (samples, variants))

# vBim_pNC: allele, vFam_pSC: samples, vBed_aNS: genotype (snp)
(vBim_pNC, vFam_pSC, vBed_aNS) = read_plink("ANM-files/anm_batch1_batch2_merged")
print( "Genotypes loaded" )


Mapping files: 100%|██████████| 3/3 [00:21<00:00,  9.89s/it]

Genotypes loaded


In [10]:
"""
Columns:

Chrom: Chromosome code (either an integer, or 'X'/'Y'/'XY'/'MT'; '0' indicates unknown) or name
SNP: Variant identifier
Cm: Position in morgans or centimorgans (safe to use dummy value of '0')
Pos: Base-pair coordinate (1-based; limited to 2^31-2)
A0: Allele 1 (corresponding to clear bits in .bed; usually minor)
A1: Allele 2 (corresponding to set bits in .bed; usually major)
"""

vBim_pNC = vBim_pNC.set_index('snp')
vBim_pNC = vBim_pNC.loc[nNs.tolist()]
vBim_pNC.reset_index(inplace=True)
vBim_pNC.head()

,snp,chrom,cm,pos,a0,a1,i
0,rs143840423,1,0.0,2936385,T,C,1881
1,rs12564456,1,0.0,2936386,G,C,1882
2,rs182722896,1,0.0,2936420,C,A,1883
3,rs72629483,1,0.0,2936678,C,T,1884
4,rs55949537,1,0.0,2936746,T,A,1885


In [12]:
import numpy as np
# FIND IN WHICH GENES GIVEN SNPS ARE
nGs = np.array(list(map(lambda s: s.replace('rna' , ''), nRs)))

# Find Chromosome locations of AddNeuroMed examples
print( "Find Chromosome locations of AddNeuroMed examples" )
from Bio import Entrez
Entrez.email = "email address"
iChromosome_nX = np.full( shape = nGs.shape,
                          fill_value = np.nan )

# Request data for all your genes  
request = Entrez.epost( "gene",
                        id = ",".join( nGs ) )
result = Entrez.read(request)
webEnv = result[ "WebEnv" ]
queryKey = result[ "QueryKey" ]
data = Entrez.esummary( db = "gene", 
                        webenv = webEnv, 
                       query_key = queryKey )
annotations = Entrez.read( data )

# Wrangle chromosome locations (uid: unique identifier)
sGeneID_0_nG = np.array( [ gene.attributes[ "uid" ] for gene in annotations[ "DocumentSummarySet" ][ "DocumentSummary" ] ] )
iChromosome_0_nG = np.array( [ gene[ "Chromosome" ] for gene in annotations[ "DocumentSummarySet" ][ "DocumentSummary" ] ] )
assert np.all( nGs == sGeneID_0_nG )

# The start and ending locations of each gene on chromosome
iChrStart_0_nG = np.array( [ gene["GenomicInfo"][0]["ChrStart"] for gene in annotations[ "DocumentSummarySet" ][ "DocumentSummary" ] ] )
iChrStop_0_nG = np.array( [ gene["GenomicInfo"][0]["ChrStop"] for gene in annotations[ "DocumentSummarySet" ][ "DocumentSummary" ] ] )

print( "Location search done.")

Find Chromosome locations of AddNeuroMed examples
Location search done.


In [13]:
# LOAD ELINK SNP_GENE FILE FROM NCBI 
import numpy as np
sSNP_Gene = np.loadtxt(fname="/Elink/snp_genes")

In [14]:
# Take snps and genes in separate numpy arrays
sSNP_nN = sSNP_Gene[:,0]
sGene_nN = sSNP_Gene[:,1]

sSNP_nN = sSNP_nN.astype('int64')
sGene_nN = sGene_nN.astype('int64')

# Sort snp names to make search faster
perm = np.argsort(sSNP_nN)
sSNP_nN = sSNP_nN[perm]
sGene_nN = sGene_nN[perm]

# SNP IDs given in the vBim_pNC['snp'] column
nNn = np.array( [ ( int(vBim_pNC['snp'][i].replace('rs', ''))) 
                 for i in range(len(vBim_pNC))]) 

# FIND IN WHICH GENES GIVEN SNPS ARE
nGeneID_0_nG = sGene_nN[np.searchsorted(sSNP_nN, nNn)]
sGeneID_0_nG = nGeneID_0_nG.astype('str')

assert(sGeneID_0_nG.shape[0] == vBim_pNC.shape[0])

In [ ]:
# CREATE INPUTS FOR EQTL ANALYSIS
genotype = np.zeros(shape = rSnp_nSRN.shape)
genotype[rSnp_nSRN == 1] = 0
genotype[rSnp_nSRN == 2] = 1
genotype[rSnp_nSRN == 3] = 2

# GENOTYPE
eqtl_genotype = pd.DataFrame(data = genotype.transpose(), columns=nSs)
eqtl_genotype.set_index(nNs, inplace=True)
eqtl_genotype.index.rename("snp", inplace=True)

# GENE EXPRESSION
eqtl_expression = pd.DataFrame(data = rRna_nSR.transpose(), columns = nSs)
eqtl_expression.set_index(nGs, inplace=True)
eqtl_expression.index.rename("id", inplace=True)

# PROBE_LOC: GENE LOCATIONS
eqtl_probe_loc = pd.DataFrame({'id': nGs,
                           'chromosome': iChromosome_0_nG,
                           'start': iChrStart_0_nG,
                           'end': iChrStop_0_nG
                          })

# SNP_LOC: SNP POSITIONS
eqtl_snp_loc = pd.DataFrame({'snp': vBim_pNC['snp'], 
                             'chrom_snp': vBim_pNC['chrom'],
                             'pos': vBim_pNC['pos'] } )

# SNP_ALLELES: SNP ALLELE
snp_alleles = pd.DataFrame({'snp': vBim_pNC['snp'], 
                            'a0': vBim_pNC['a0'],
                            'a1': vBim_pNC['a1'] } )

# MAPPING from SNP to GENE
snp_gene_data = {'snp': nNs, 'gene': sGeneID_0_nG}
snp_gene = pd.DataFrame(data = snp_gene_data)

assert( eqtl_genotype.shape[1] == eqtl_expression.shape[1] )
assert( eqtl_genotype.shape[0] == eqtl_snp_loc.shape[0] )
assert( eqtl_expression.shape[0] == eqtl_probe_loc.shape[0] )

print("eQTL inputs in pandas format created.")

# eQTL: STORE IN .CSV FORMAT
expression_csv = eqtl_expression.to_csv("../eQTL + LD analysis/AddNeuroMed/eqtl_ANM_expression.csv",
                                        sep=",",
                                        header=True)

genotype_csv = eqtl_genotype.to_csv("../eQTL + LD analysis/AddNeuroMed/eqtl_ANM_genotype.csv",
                                        sep=",",
                                        header=True)

probe_loc_csv = eqtl_probe_loc.to_csv("../eQTL + LD analysis/AddNeuroMed/eqtl_ANM_probe_loc.csv",
                                        sep=",",
                                        header=True)

snp_loc_csv = eqtl_snp_loc.to_csv("../eQTL + LD analysis/AddNeuroMed/eqtl_ANM_snp_loc.csv",
                                        sep=",",
                                        header=True)

print("eQTL inputs stored in CSV format.")


# HELPER VARIABLES
subjects_csv = pd.DataFrame(data=nSs).to_csv("../eQTL + LD analysis/AddNeuroMed/eqtl_ANM_subjects.csv",
                                   sep =",",
                                   header=True)

snps_csv = pd.DataFrame(data=nNs).to_csv("../eQTL + LD analysis/AddNeuroMed/eqtl_ANM_snps.csv",
                                   sep =",",
                                   header=True)

genes_csv = pd.DataFrame(data=nGs).to_csv("../eQTL + LD analysis/AddNeuroMed/eqtl_ANM_genes.csv",
                                   sep =",",
                                   header=True)

snp_gene_csv = pd.DataFrame(data=snp_gene).to_csv("../eQTL + LD analysis/AddNeuroMed/eqtl_ANM_snp_gene.csv",
                                   sep =",",
                                   header=True)

snp_alleles_csv = pd.DataFrame(data=snp_alleles).to_csv("../eQTL + LD analysis/AddNeuroMed/eqtl_snp_alleles.csv",
                                   sep =",",
                                   header=True)

print("eQTL input helpers stored in CSV format.")

In [9]:
# SAVE DATA
# Save all the transformed data into a pickle, such that it can be more easily loaded from Python

# Save data into Python file
import pickle
r( 'rm( list = ls( ) )' ) #Free space
with open( 'loadData.pickle', 'wb' ) as f:
    pickle.dump( rRna_nSR, f, pickle.HIGHEST_PROTOCOL )
    pickle.dump( sDem_nSD, f, pickle.HIGHEST_PROTOCOL )
    pickle.dump( rSnp_nSN, f, pickle.HIGHEST_PROTOCOL )
    pickle.dump( sGenesAD_nG, f, pickle.HIGHEST_PROTOCOL )
    pickle.dump( nNs_lRlN, f, pickle.HIGHEST_PROTOCOL )
    pickle.dump( nPairs_lPlC, f, pickle.HIGHEST_PROTOCOL )
    pickle.dump( nSs, f, pickle.HIGHEST_PROTOCOL )
    pickle.dump( nRs, f, pickle.HIGHEST_PROTOCOL )
    pickle.dump( nDs, f, pickle.HIGHEST_PROTOCOL )
    pickle.dump( nNs, f, pickle.HIGHEST_PROTOCOL )
    print( 'Data saved into pickle.' )

Data saved into pickle.
